## Closed Loop Simulation Notebook

This notebook breaks down the closed loop simulation in different jupyter notebook cells. The aim of this notebook is to ease the debugging and development of the different packages involved in the Optical Feedback Control loop.

This notebook was last run on Aug. 12th 2021. 
Details:
- Machine size: 32 CPUs, 16 cores per socket (2 sockets).
- Stack release: 2021_13, Last verified to run 2021-08-12
- ts_phosim release: v1.4.0


### Required python packages

In [2]:
import argparse
import logging
import os
import sys
import time
import shutil

from lsst.ts.phosim.CloseLoopTask import CloseLoopTask
from lsst.ts.phosim.telescope.TeleFacade import TeleFacade

In [3]:
from tqdm import tqdm

### Default arguments and initializations

Initialize default arguments to run the closed loop and initialize the ClosedLoopTask

In [4]:
%%time

# Set the parser
parser = argparse.ArgumentParser(
    description="Run AOS closed-loop simulation (default is amplifier files)."
)
parser = CloseLoopTask.setDefaultParser(parser)
parser = CloseLoopTask.setImgParser(parser)

# Get the default arguments
sys.argv = ['-f']
args = parser.parse_args()

# Print default arguments
print(args)

logger = logging.getLogger()
logger.setLevel(args.log_level)

# Initialize the ClosedLoopTask
closeLoopTask = CloseLoopTask()

Namespace(boresightDeg=[0, 0], clobber=False, eimage=False, filterType='', inst='comcam', iterNum=5, log_level=20, m1m3FErr=0.05, numOfProc=1, output='', rotCam=0.0, skyFile='')
CPU times: user 3.96 ms, sys: 48 µs, total: 4.01 ms
Wall time: 3.46 ms


### Set paths and arguments

Customize the following arguments to run the simulation you are interested in. If not stated in the following code cell, the arguments are taken to be the default ones. Note that this paths should be changed if another user is running them and wants to change the directory addresses.

In [5]:
PHOSIMPATH = "/project/gmegias/aos/lsst_stack/phosim_syseng4/"
AOCLCOUTPUTPATH = "/project/gmegias/aos/lsst_stack/ts_phosim/output/"
os.environ["PHOSIMPATH"] = PHOSIMPATH
os.environ["AOCLCOUTPUTPATH"] = AOCLCOUTPUTPATH

In [6]:
args.inst = 'comcam' 
args.numOfProc = 32 
args.boresightDeg = [0.03, -0.02]
args.skyFile = '/project/gmegias/aos/lsst_stack/ts_phosim/tests/testData/sky/skyComCam.txt'
args.output = '/project/gmegias/aos/perturbations/imgCloseLoop_test12/'

if os.path.exists(args.output):
    shutil.rmtree(args.output)

### Intializations and initial configurations

First, initialize the variables from the arguments dictionary.

In [7]:
boresight = args.boresightDeg
rotCamInDeg = args.rotCam
useEimg = args.eimage
m1m3ForceError = args.m1m3FErr
numPro = args.numOfProc
iterNum = args.iterNum
doErsDirCont = args.clobber
pathSkyFile = args.skyFile

Check and set the required configurations to run the simulation loop.

In [8]:
%%time

# Check the input arguments
camType, instName = closeLoopTask.getCamTypeAndInstName(args.inst)
filterType = closeLoopTask.getFilterType(args.filterType)
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)

if doErsDirCont:
    closeLoopTask.eraseDirectoryContent(baseOutputDir)

closeLoopTask.checkBoresight(boresight, args.skyFile)

closeLoopTask.assignImgType(useEimg)

# Configure the components
closeLoopTask.configSkySim(instName, pathSkyFile = args.skyFile, starMag=15)

pathIsrDir = closeLoopTask.createIsrDir(baseOutputDir)
closeLoopTask.configWepCalc(
    camType, pathIsrDir, filterType, boresight, rotCamInDeg, useEimg=useEimg
)

closeLoopTask.configOfcCalc(instName)
closeLoopTask.configPhosimCmpt(
    filterType, rotCamInDeg, m1m3ForceError, numPro
)

# Set the defocal distance for WEP calculator based on the setting
# file in the telescope
closeLoopTask.setWepCalcWithDefocalDist()

/project/gmegias/aos/lsst_stack/ts_wep/python/lsst/ts/wep/bsc/WcsSol.py:45: FutureWarning: Gen2 Butler has been deprecated (LsstCamMapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  self._camera = obs_lsst.lsstCamMapper.LsstCamMapper().camera


CPU times: user 2min 43s, sys: 3.02 s, total: 2min 46s
Wall time: 2min 47s


### Generate butler gen3 repository.

In [9]:
%%time

# generate bluter gen3 repo if needed
butlerRootPath = os.path.join(baseOutputDir, "phosimData")
if closeLoopTask.useCcdImg():
    closeLoopTask.generateButler(butlerRootPath, instName)
    closeLoopTask.generateRefCatalog(
        instName = instName,
        butlerRootPath = butlerRootPath,
        pathSkyFile = pathSkyFile,
    )
    
closeLoopTask.phosimCmpt.tele.setInstName(camType)

CPU times: user 36.7 ms, sys: 65.1 ms, total: 102 ms
Wall time: 3min 15s


### Simulation loop initialization

Initialize the state of the telescope to run the simulation. This cell also sets common file and directory names required for the closed loop simulation.

In [10]:
%%time

# Set the telescope state to be the same as the OFC
state0 = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
closeLoopTask.phosimCmpt.setDofInUm(state0)

# Get the list of referenced sensor name (field positions)
refSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
refSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)

# Common file and directory names
opdZkFileName = "opd.zer"
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
dofInUmFileName = "dofPertInNextIter.mat"
fwhmItersFileName = "fwhmIters.png"

# Specific file names to the amplifier/eimage
wfsZkFileName = "wfs.zer"

CPU times: user 84 ms, sys: 6.02 ms, total: 90 ms
Wall time: 88.1 ms


### Define Phosim initalization functions

In [13]:

def _writePertAndCmdFiles(self, cmdSettingFileName, cmdFileName):
    """Write the physical perturbation and command files.
    Parameters
    ----------
    cmdSettingFileName : str
        Physical command setting file name.
    cmdFileName : str
        Physical command file name.
    Returns
    -------
    str
        Command file path.
    """

    # Write the perturbation file
    pertCmdFileName = "pert.cmd"

    pertCmdFilePath = os.path.join(self.outputDir, pertCmdFileName)
    tele = TeleFacade()
    if not os.path.exists(pertCmdFilePath):
        tele.writePertBaseOnConfigFile(
            self.outputDir,
            seedNum=self.seedNum,
            m1m3ForceError=self.m1m3ForceError,
            saveResMapFig=True,
            pertCmdFileName=pertCmdFileName,
        )

    # Write the physical command file
    cmdSettingFile = os.path.join(self.configDir, "cmdFile", cmdSettingFileName)
    cmdFilePath = os.path.join(self.outputDir, cmdFileName)
    if not os.path.exists(cmdFilePath):
        self.tele.writeCmdFile(
            self.outputDir,
            cmdSettingFile=cmdSettingFile,
            pertFilePath=pertCmdFilePath,
            cmdFileName=cmdFileName,
        )

    return cmdFilePath

def getOpdArgsAndFilesForPhoSim(
        phosimCmpt,
        instName,
        cmdFileName="opd.cmd",
        instFileName="opd.inst",
        logFileName="opdPhoSim.log",
        cmdSettingFileName="opdDefault.cmd",
        instSettingFileName="opdDefault.inst",
    ):
        """Get the OPD calculation arguments and files for the PhoSim
        calculation.
        OPD: optical path difference.
        Parameters
        ----------
        instName : `str`
            Instrument name.
        cmdFileName : str, optional
            Physical command file name. (the default is "opd.cmd".)
        instFileName : str, optional
            OPD instance file name. (the default is "opd.inst".)
        logFileName : str, optional
            Log file name. (the default is "opdPhoSim.log".)
        cmdSettingFileName : str, optional
            Physical command setting file name. (the default is
            "opdDefault.cmd".)
        instSettingFileName : str, optional
            Instance setting file name. (the default is "opdDefault.inst".)
        Returns
        -------
        str
            Arguments to run the PhoSim.
        """

        # Set the weighting ratio and field positions of OPD
        phosimCmpt.metr.setWgtAndFieldXyOfGQ(instName)

        # Write the command file
        cmdFilePath = _writePertAndCmdFiles(phosimCmpt, cmdSettingFileName, cmdFileName)

        # Write the instance file
        instSettingFile = phosimCmpt._getInstSettingFilePath(instSettingFileName)

        instFilePath = phosimCmpt.tele.writeOpdInstFile(
            phosimCmpt.outputDir,
            phosimCmpt.metr,
            instSettingFile=instSettingFile,
            instFileName=instFileName,
        )

        # Get the argument to run the PhoSim
        argString = phosimCmpt._getPhoSimArgs(logFileName, instFilePath, cmdFilePath)

        return argString
    
getOpdArgsAndFilesForPhoSim(closeLoopTask.phosimCmpt, instName)

'/project/gmegias/aos/perturbations/imgCloseLoop_test12/iter0/pert/opd.inst -i comcam -e 1 -c /project/gmegias/aos/perturbations/imgCloseLoop_test12/iter0/pert/opd.cmd -p 32 -o /project/gmegias/aos/perturbations/imgCloseLoop_test12/iter0/img > /project/gmegias/aos/perturbations/imgCloseLoop_test12/iter0/img/opdPhoSim.log 2>&1'

### Run simulation for each iteration

In [14]:
# Do the iteration
obsId = 9006000

for iterCount in tqdm(range(iterNum)):
    print(iterCount)

    # Set the observation Id
    closeLoopTask.phosimCmpt.setSurveyParam(obsId=obsId)

    # The iteration directory
    iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

    # Set the output directory
    outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
    closeLoopTask.phosimCmpt.setOutputDir(outputDir)

    # Set the output image directory
    outputImgDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
    closeLoopTask.phosimCmpt.setOutputImgDir(outputImgDir)

    # Generate the OPD image
    print("Preparing PhoSim....")
    t0 = time.time()
    argString = getOpdArgsAndFilesForPhoSim(closeLoopTask.phosimCmpt, instName)
    ttl_time = time.time() - t0
    print("PhoSim prepared, took {} seconds.".format(ttl_time))
    
    # Run Phosimd
    print("Running PhoSim....")
    t0 = time.time()
    closeLoopTask.phosimCmpt.runPhoSim(argString)
    ttl_time = time.time() - t0
    print("PhoSim run, took {} seconds.".format(ttl_time))

    # Analyze the OPD data
    # Rotate OPD in the reversed direction of camera
    print("Analyzing OPD data....")
    t0 = time.time()
    closeLoopTask.phosimCmpt.analyzeOpdData(
        instName,
        zkFileName=opdZkFileName,
        rotOpdInDeg=-rotCamInDeg,
        pssnFileName=opdPssnFileName,
    )
    ttl_time = time.time() - t0
    print("OPD data analyzed, took {} seconds.".format(ttl_time))
    
    # Get the PSSN from file
    pssn = closeLoopTask.phosimCmpt.getOpdPssnFromFile(opdPssnFileName)
    print("Calculated PSSN is %s." % pssn)

    # Get the GQ effective FWHM from file
    gqEffFwhm = closeLoopTask.phosimCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)
    print("GQ effective FWHM is %.4f." % gqEffFwhm)

    # Set the FWHM data
    print("Setting FWHM data...")
    t0 = time.time()
    fwhm, sensor_id = closeLoopTask.phosimCmpt.getListOfFwhmSensorData(
        opdPssnFileName, refSensorIdList
    )

    closeLoopTask.ofcCalc.set_fwhm_data(fwhm, sensor_id)
    ttl_time = time.time() - t0
    print("FWHM data set, took {} seconds.".format(ttl_time))

    # Generate the sky images and calculate the wavefront error
    print("Generating sky images and calculating WF error...")
    t0 = time.time()
    if closeLoopTask.useCcdImg():
        listOfWfErr = closeLoopTask._calcWfErrFromImg(
            obsId, butlerRootPath = butlerRootPath, instName=instName, snap=0
        )
    else:
        # Simulate to get the wavefront sensor data from WEP
        listOfWfErr = closeLoopTask.phosimCmpt.mapOpdDataToListOfWfErr(
            opdZkFileName, refSensorIdList
        )
    ttl_time = time.time() - t0
    print("Sky images and WF error generated. Took {} seconds.".format(ttl_time))

    # Record the wavefront error with the same order as OPD for the
    # comparison
    print('Recording wavefront error...')
    t0 = time.time()
    if closeLoopTask.useCcdImg():
        closeLoopTask.phosimCmpt.reorderAndSaveWfErrFile(
            listOfWfErr,
            refSensorNameList,
            getCamera(instName),
            zkFileName=wfsZkFileName,
        )
    ttl_time = time.time() - t0
    print('Wavefront error recorded. Took {} seconds.'.format(ttl_time))
        
    # Calculate the DOF
    print('Calculating DOF...')
    t0 = time.time()
    wfe = np.array(
        [sensor_wfe.getAnnularZernikePoly() for sensor_wfe in listOfWfErr]
    )
    sensor_ids = np.array(
        [sensor_wfe.getSensorId() for sensor_wfe in listOfWfErr]
    )
    ttl_time = time.time() - t0
    print('DOF computed. Took {} seconds.'.format(ttl_time))

    print('Calculating corrections...')
    t0 = time.time()
    closeLoopTask.ofcCalc.calculate_corrections(
        wfe=wfe,
        field_idx=sensor_ids,
        filter_name=str(filterType),
        gain=-1,
        rot=rotCamInDeg,
    )
    ttl_time = time.time() - t0
    print('Corrections computed. Took {} seconds.'.format(ttl_time))

    # Set the new aggregated DOF to phosimCmpt
    print('Setting and saving new DOF...')
    t0 = time.time()
    dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
    closeLoopTask.phosimCmpt.setDofInUm(dofInUm)

    # Save the DOF file
    closeLoopTask.phosimCmpt.saveDofInUmFileForNextIter(
        dofInUm, dofInUmFileName=dofInUmFileName
    )
    ttl_time = time.time() - t0
    print('New DOF saved. Took {} seconds.'.format(ttl_time))

    # Add the observation ID by 10 for the next iteration
    obsId += 10

  0%|          | 0/5 [00:00<?, ?it/s]

0
Preparing PhoSim....
PhoSim prepared, took 0.013274431228637695 seconds.
Running PhoSim....
PhoSim run, took 108.8821747303009 seconds.
Analyzing OPD data....
OPD data analyzed, took 4.577322483062744 seconds.
Calculated PSSN is [0.98029371 0.98606982 0.98029371 0.98606982 0.98372946 0.98606982
 0.98159617 0.98606982 0.9802937 ].
GQ effective FWHM is 0.0844.
Setting FWHM data...
FWHM data set, took 0.002550363540649414 seconds.
Generating sky images and calculating WF error...


  0%|          | 0/5 [15:29<?, ?it/s]


RuntimeError: Error running: pipetask run -b /project/gmegias/aos/perturbations/imgCloseLoop_test12/phosimData -i refcats,LSSTComCam/raw/all,LSSTComCam/calib --instrument lsst.obs.lsst.LsstComCam --register-dataset-types --output-run ts_phosim_9006001 -p comcamPipeline.yaml -d "exposure IN (4021123106001, 4021123106002)" -j 2

### Summarize the FWHM and plot pssn profiles

In [ ]:
# Summarize the FWHM
pssnFiles = [
    os.path.join(
        baseOutputDir,
        "%s%d" % (iterDefaultDirName, num),
        outputImgDirName,
        opdPssnFileName,
    )
    for num in range(iterNum)
]
saveToFilePath = os.path.join(baseOutputDir, fwhmItersFileName)
plotFwhmOfIters(pssnFiles, saveToFilePath=saveToFilePath)